# Тестирование сервиса в ручную

In [1]:
from pathlib import Path

DATA_PATH = Path('../../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = DATA_PATH / Path('cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

LOG_DATA = Path('../../logs/')
LOG_DATA.mkdir(parents=True, exist_ok=True)

import pandas as pd

pd.set_option('display.max_colwidth', 500) 

In [2]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_path)

In [3]:
from app.settings import settings
from app.models.openai import OpenAITextModel, OpenAIImageModel, OpenAIAudioTranscribeModel

# Проверим API ключ
print(f"OpenAI API Key загружен: {'да' if settings.OPENAI_API_KEY else 'нет'}")


OpenAI API Key загружен: да


# Чат

In [4]:
# llm_openai = OpenAITextModel(api_key=settings.OPENAI_API_KEY, model_name="gpt-4.1-nano")


In [5]:
# messages = [{"role": "user", "content": "Привет, как тебя зовут?"}]
# response = llm_openai.generate(messages=messages)

# print("Ответ модели:")
# print(response)


In [6]:
# messages = [{"role": "user", "content": "Как твои дела?"}]

# print("Потоковый ответ:")
# for chunk in llm_openai.generate_stream(messages=messages):
#     print(chunk, end="", flush=True)


In [7]:
# from app.models.yandex import YandexChatModel

# llm_yandex = YandexChatModel(
#     auth=settings.YANDEX_API_KEY,
#     folder_id=settings.YANDEX_FOLDER_ID,
#     model_name="yandexgpt-lite"
# )

In [8]:
# messages = [{"role": "user", "text": "Привет, как тебя зовут?"}]
# response = llm_yandex.generate(messages=messages)

# print("Ответ YandexGPT:")
# print(response)

In [9]:
# messages = [{"role": "user", "text": "Как твои дела?"}]

# response = llm_yandex.generate_stream(messages=messages)
# response

In [10]:
# from app.services.token_manager import TokenManager

# tm = TokenManager()
# text = "Привет, мир!"
# print(f"Текст: {text}")
# print(f"Токенов: {tm.count_tokens(text, model_name='gpt-4.1-nano')}")


# Транскрибация

In [ ]:
import io
from pathlib import Path

from app.models.whisper import WhisperService
from app.settings import settings

audio_model = WhisperService(
    model_name="tiny",
    # model_name="base",
    # model_name="dvislobokov/faster-whisper-large-v3-turbo-russian",
    # model_name="Systran/faster-whisper-large-v3",
    # model_name="Zoont/faster-whisper-large-v3-turbo-int8-ct2",
    device="cpu",
    # compute_type="float16",
)

audio_path = Path("test.wav")

d:\Projects\env_ml\.venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
with open(audio_path, "rb") as f:
    audio_stream = io.BytesIO(f.read())


result_text = audio_model.generate(
    audio_stream,
    language="ru",
    audio_format="wav",
    # task="transcribe",
    # beam_size=5,
    # temperature=0.1,
)

print("Результат транскрипции:")
print(result_text)

2025-08-21 18:19:31 - faster_whisper - INFO - Processing audio with duration 00:07.620
Результат транскрипции:
Шала Саша, Пошасы и Сасала Сушку.


In [ ]:
# import io

# audio_model = OpenAIAudioTranscribeModel(
#     api_key=settings.OPENAI_API_KEY,
#     model_name="gpt-4o-mini-transcribe"
# )

# audio_path = Path("test.wav")
# with open(audio_path, "rb") as f:
#     audio_stream = io.BytesIO(f.read())

In [ ]:
# result_text = audio_model.generate(
#     audio_stream,
#     filename=audio_path.name,
#     language="ru"
# )

# print("Результат транскрипции:")
# print(result_text)


In [ ]:
# import io
# from pathlib import Path

# from app.models.yandex import YandexSpeechToTextModel
# from app.settings import settings

# audio_model = YandexSpeechToTextModel(
#     auth=settings.YANDEX_API_KEY,
#     folder_id=settings.YANDEX_FOLDER_ID,
#     model_name="general",
#     language="ru-RU"
# )
# audio_path = Path("test.wav")
# with open(audio_path, "rb") as f:
#     audio_stream = io.BytesIO(f.read())

In [ ]:
# result_text = audio_model.generate(
#     audio_stream,
#     filename=audio_path.name,
#     enable_automatic_punctuation=True
# )

# print("Результат транскрипции:")
# print(result_text)

# Аудео

# Изображения

In [ ]:
# import base64
# from pathlib import Path
# from app.models.openai import OpenAIImageModel
# from app.settings import settings

# image_model = OpenAIImageModel(
#     api_key=settings.OPENAI_API_KEY,
#     model_name="gpt-image-1"
# )

# prompt = "Футуристический город на закате"
# result_images = image_model.generate(
#     prompt=prompt,
#     n=1,
#     size="1024x1024",
#     quality="low"
# )

In [ ]:

# output_dir = Path("generated_images")
# output_dir.mkdir(exist_ok=True)

# for idx, img_b64 in enumerate(result_images, start=1):
#     img_bytes = base64.b64decode(img_b64)
#     img_path = output_dir / f"image_{idx}.png"
#     with open(img_path, "wb") as f:
#         f.write(img_bytes)
#     print(f"✅ Сохранено: {img_path}")


In [ ]:
# from app.models.yandex import YandexArtModel
# from app.settings import settings

# # Инициализация модели
# art_model = YandexArtModel(
#     auth=settings.YANDEX_API_KEY,
#     folder_id=settings.YANDEX_FOLDER_ID,
#     model_name="yandex-art"
# )

In [ ]:
# prompt = "Нарисуй красную кошку в стиле Миядзаки"

# result = art_model.generate(prompt, output_path="./cat.jpeg", width_ratio=1, height_ratio=1, seed=42)

# print("Изображение успешно сгенерировано и сохранено в ./cat.jpeg")
# print("Версия модели:", result.model_version)
# print("Размер байтов:", len(result.image_bytes))